In [1]:
import pandas as pd
import numpy as np

In [43]:
df = pd.read_excel('value structure 2022-2024.xlsx')
df_ego = pd.read_excel('/Users/jiazhengtian/Desktop/Research/Paul Ingram_25.2.21/All Values/combined id_ego thru fall 24.xlsx')

In [44]:
df.head()

,id_ego,Person Name,From,To,NoMeaningfulEdges,Operations,Data
0,7023.0,Josh Adler,strength,ecstasy,1.0,NaN,2022 A
1,7023.0,Josh Adler,strength,discovery,1.0,NaN,2022 A
2,7023.0,Josh Adler,strength,bravery,1.0,NaN,2022 A
3,7023.0,Josh Adler,strength,calm,1.0,NaN,2022 A
4,7023.0,Josh Adler,calm,connected to the universe,1.0,NaN,2022 A


In [45]:
missing_id = df[df['id_ego'].isna()][['id_ego', 'Person Name']].drop_duplicates()

In [46]:
missing_id.head()

,id_ego,Person Name
205,NaN,Jake Recker
310,NaN,Paulo Humberto Varnieri
318,NaN,Jake Weiss
327,NaN,Hugo Fausto Torres-Fetsco
407,NaN,Lindsey Gitlin


In [47]:
df_ego.head()

,id_ego,SubjectEmail,Last name,First Name,full name
0,1,mantoniou22@gsb.columbia.edu,NaN,NaN,Michael Antoniou
1,2,tatwood22@gsb.columbia.edu,NaN,NaN,Taylor Atwood
2,3,jbaumann22@gsb.columbia.edu,NaN,NaN,Joss Baumann
3,4,mbennett22@gsb.columbia.edu,NaN,NaN,Michael Bennett
4,5,paul.e.bird@columbia.edu,NaN,NaN,Paul Bird


# Name Parsing

In [48]:
def name_parser(full_name):
    if pd.isna(full_name):
        return ('','')
    else:
        name_parts = full_name.split()
        return name_parts[0], name_parts[-1]


In [49]:
missing_id['first_name'], missing_id['last_name'] = zip(*missing_id['Person Name'].apply(name_parser))
df_ego['first_name'], df_ego['last_name'] = zip(*df_ego['full name'].apply(name_parser))

In [50]:
missing_id.head()

,id_ego,Person Name,first_name,last_name
205,NaN,Jake Recker,Jake,Recker
310,NaN,Paulo Humberto Varnieri,Paulo,Varnieri
318,NaN,Jake Weiss,Jake,Weiss
327,NaN,Hugo Fausto Torres-Fetsco,Hugo,Torres-Fetsco
407,NaN,Lindsey Gitlin,Lindsey,Gitlin


In [51]:
df_ego.head()

,id_ego,SubjectEmail,Last name,First Name,full name,first_name,last_name
0,1,mantoniou22@gsb.columbia.edu,NaN,NaN,Michael Antoniou,Michael,Antoniou
1,2,tatwood22@gsb.columbia.edu,NaN,NaN,Taylor Atwood,Taylor,Atwood
2,3,jbaumann22@gsb.columbia.edu,NaN,NaN,Joss Baumann,Joss,Baumann
3,4,mbennett22@gsb.columbia.edu,NaN,NaN,Michael Bennett,Michael,Bennett
4,5,paul.e.bird@columbia.edu,NaN,NaN,Paul Bird,Paul,Bird


In [60]:
id_lookup = df_ego.dropna(subset=['id_ego', 'full name', 'first_name', 'last_name'])
id_lookup['id_ego'] = id_lookup['id_ego'].astype(str)

lookup_dict = {
    (row['last_name'].strip().lower(), row['first_name'].strip().lower()): (row['id_ego'], row['full name'])
    for _, row in id_lookup.iterrows()
}

/var/folders/pj/csflcfl13v78mjmgbc3k8k3r0000gn/T/ipykernel_30165/3607866485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_lookup['id_ego'] = id_lookup['id_ego'].astype(str)


In [62]:
def matching(row):
    key = (row['last_name'].strip().lower(), row['first_name'].strip().lower())
    match = lookup_dict.get(key, (None, None))
    return pd.Series(match, index=['matched_id', 'matched_full_name'])

missing_id[['matched_id', 'matched_full_name']] = missing_id.apply(matching, axis=1)

In [58]:
missing_id.isna().sum()

id_ego         104
Person Name      0
first_name       0
last_name        0
matched_id      65
dtype: int64

In [63]:
missing_id.to_excel('missing_id_matched.xlsx', index=False)